In [3]:
import re, math, pickle, time, os
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.rcParams['figure.dpi']= 200
import sklearn.model_selection
import torch
from torch import optim, nn
from torch.autograd import Variable

import model_to_human_decision_torch 
from model_to_human_decision_torch import get_parameters_across_models

## load task data

In [4]:
task_data_csv='data_exp1_cumulative.csv'

In [5]:
def prepare_tidy_protocol(csv_path,do_remove_within_subject_repetitions=True):
    df=pd.read_csv(csv_path)
    
    df=df.loc[df.Response.notna(),:] #drop non-response lines
    df=df.rename(columns={'source':'source_set 1'}) # fix a mistake in model naming
    df=df.rename(columns={'counterbalance-o1ql':'set_num'})

    # transform set columns to trial columns
    p = re.compile('(.+)_set \d+')
    set_columns=[s for s in list(df.columns) if p.match(s)]
    columns_to_keep=['Participant External Session ID','Event Index','Reaction Time','Response','set_num']+set_columns
    columns_to_drop=list(np.setdiff1d(df.columns,columns_to_keep))
    df=df.drop(columns=columns_to_drop)
    columns_to_build=list(np.unique([p.findall(s)[0] for s in set_columns]))
    for index,row in df.iterrows():
        cur_set=row['set_num']
        for c in columns_to_build:
            df.loc[index,c]=df.loc[index,c+'_'+cur_set]
    df=df.drop(columns=set_columns+['set_num'])

    # make sure the sentence pairs are alphabetically sorted
    def sort_sentence_pairs(df1):    
        flip_dict={'sentence1':'sentence2','sentence2':'sentence1',
                   'sentence1_model':'sentence2_model','sentence2_model':'sentence1_model'}
        df2=df1.copy()    
        for index, row in df2.iterrows():
            if row['sentence1']>row['sentence2']: # a flip is needed
                for old_col, new_col in flip_dict.items():
                    df2.loc[index,new_col]=df1.loc[index,old_col]
        return df2
    df=sort_sentence_pairs(df)

    # separate models and levels
    p=re.compile('(.+)_(\d+)')
    df['sentence1_model_name']=[p.findall(s)[0][0] for s in df['sentence1_model']]
    df['sentence2_model_name']=[p.findall(s)[0][0] for s in df['sentence2_model']]
    df['sentence1_model_level']=[int(p.findall(s)[0][1]) for s in df['sentence1_model']]
    df['sentence2_model_level']=[int(p.findall(s)[0][1]) for s in df['sentence2_model']]
    df=df.drop(columns=['sentence1_model','sentence2_model'])

    # mark choice indecis
    for index,row in df.iterrows():
        df.loc[index,'Choice']=[row.sentence1, row.sentence2].index(row.Response)

    df['sentence_pair_id']=[s1+'_'+s2 for s1,s2 in zip(df.sentence1, df.sentence2)]

    # renumber subjects
    df.drop(columns='subject') # there's some issue here    
    uq_subjects, ind, unique_inverse = np.unique(df['Participant External Session ID'], return_index=True,return_inverse=True)
    uq_subjects=list(uq_subjects[np.argsort(ind)]) # unique subjects by order of appearance    
    df['subject']=[uq_subjects.index(s) for s in df['Participant External Session ID']]
                   
    # optionally, remove all but first occurance of each sentence pair within a subject    
    if do_remove_within_subject_repetitions:
        def remove_within_subject_repetitions(df1):
            # keep only the first appearance of each sentence pair within a subject
            _,unique_indices=np.unique(df1.sentence_pair_id,return_index=True)
            df2=df1.loc[np.in1d(np.arange(len(df1)),unique_indices),:]    
            return df2
        old_length=len(df)
        df=df.groupby('subject').apply(remove_within_subject_repetitions)
        print("removed {} within-subject repeating trials.".format(old_length-len(df)))
        
    df=df.set_index(['subject','trial']).sort_values(by=['subject','trial'],axis=0).reset_index()
    
    # a final sanity check: 
    assert len(np.unique(df['sentence_pair_id']))==len(df.drop_duplicates(subset=['sentence1','sentence2'])), 'number of sentences pairs must match number of sentence pair ids'
    
    return df
protocol=prepare_tidy_protocol(task_data_csv)


/home/tal/anaconda3/envs/contstimlang/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


removed 70 within-subject repeating trials.


In [6]:
n_total_trials=len(protocol)
n_sentence_pairs=len(np.unique(protocol['sentence_pair_id']))
n_subjects=len(np.unique(protocol['Participant External Session ID']))
print("loaded {} total trials with {} sentence pairs and {} subjects.".format(n_total_trials,n_sentence_pairs,n_subjects))

loaded 2570 total trials with 605 sentence pairs and 24 subjects.


In [7]:
protocol

,subject,trial,Event Index,Participant External Session ID,Reaction Time,Response,sentence1,sentence2,source,sentence1_model_name,sentence2_model_name,sentence1_model_level,sentence2_model_level,Choice,sentence_pair_id
0,0,1.0,4,5f89a28900ebf4068239ad58,5812.220000,What public schools teach their students is co...,I can not long conceal my dreadful sin,What public schools teach their students is co...,internet,xlm,xlm,1,0,1.0,I can not long conceal my dreadful sin_What pu...
1,0,2.0,6,5f89a28900ebf4068239ad58,5300.000000,And that consists of a number of factors,And that consists of a number of factors,Birmingham but the heavy duty cell one and,generator,electra,electra,9,5,0.0,And that consists of a number of factors_Birmi...
2,0,3.0,8,5f89a28900ebf4068239ad58,9964.815000,Dad of a stagnant fermentation shrimp of doubts,Dad of a stagnant fermentation shrimp of doubts,Mel causal that foothold overflowed paid Cpl a...,generator,bilstm,bilstm,5,0,0.0,Dad of a stagnant fermentation shrimp of doubt...
3,0,4.0,10,5f89a28900ebf4068239ad58,11878.520000,Brisbane corp distant and finished commandos c...,Brisbane corp distant and finished commandos c...,The frustrating and bananas and the modificati...,generator,bigram,bigram,2,6,0.0,Brisbane corp distant and finished commandos c...
4,0,5.0,12,5f89a28900ebf4068239ad58,12686.870000,Listening those studied bails heater understan...,Hours into cults cables help arrive in mounts,Listening those studied bails heater understan...,generator,lstm,lstm,5,2,1.0,Hours into cults cables help arrive in mounts_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565,23,106.0,214,5fde554b85dbbb5cf9a1d168,1467.500000,List of people in America who hate it,Blog negative blog shelve grammar represents c...,List of people in America who hate it,generator,xlm,xlm,3,9,1.0,Blog negative blog shelve grammar represents c...
2566,23,107.0,216,5fde554b85dbbb5cf9a1d168,2621.625001,Lions in his most charming little extraterrest...,Forces bucks hometown donors Berger complicati...,Lions in his most charming little extraterrest...,generator,trigram,trigram,1,5,1.0,Forces bucks hometown donors Berger complicati...
2567,23,108.0,218,5fde554b85dbbb5cf9a1d168,2137.399999,Immediately after completing algorithms the ga...,Danny Louvre shrouded typically flees offendin...,Immediately after completing algorithms the ga...,generator,roberta,roberta,1,6,1.0,Danny Louvre shrouded typically flees offendin...
2568,23,109.0,220,5fde554b85dbbb5cf9a1d168,1881.740000,The man sleeps with the second solid object,Hart bitter motions Toledo going whacking emer...,The man sleeps with the second solid object,generator,xlm,xlm,1,8,1.0,Hart bitter motions Toledo going whacking emer...


## Evaluate negative log-likelihood of choice predictions given as pandas dataframe

In [8]:
def evaluate_NLL(subject_data,predictions,mode='sum',deal_with_missing_predictions='omit',return_non_agg_NLL=False):
    """ evaluate the negative-log-likelihood of human response data given model predictions.
    
    subject_data (pd.DataFrame) a dataframe such as the one returned from prepare_tidy_protocol. Should contain 'subject' (str), 'sentence_pair_id' (str) and 'Choice' (either 0 or 1).
    predictions (pd.DataFrame) model predictions. 
        Should contain sentence_pair_id (str), and either 'choice_0_prob' and 'choice_1_prob' or 'choice_0_log_prob' and 'choice_1_log_prob': 
        the (log) probabilitiy of choosing sentence 0 or the other one.
        the predictions dataframe can contain also a 'subject' column for subject specific-predictions
    mode (str) 'sum' or 'mean' (how NLL is aggregeated).
    deal_with_missing_predictions (str) 'omit' - deletes trials with no predictions. uniform predictions - uses a p=0.5 prediction.
    return_non_agg_NLL (boolean)
    
    returns:
    if return_non_agg_NLL is True, (aggregated dataframe, original dataframe)
    else, only the aggregated dataframe
    both return dataframes contain a new columns NLL - negative-log-likelihood
    
    """
    if 'subject' in predictions.columns: # predictions are subject-specific
        df=subject_data.merge(predictions,how='left',on=['sentence_pair_id','subject'])
    else: # same predictions for all subjects
        df=subject_data.merge(predictions,how='left',on=['sentence_pair_id'])
    c=np.asarray(subject_data.Choice,dtype=float)
    
    if 'choice_0_log_prob' in predictions.columns and 'choice_1_log_prob' in predictions.columns:
        log_p=np.where(c==0,df.choice_0_log_prob,df.choice_1_log_prob)
        log_p=np.where(np.isnan(c),np.nan,log_p)
    elif 'choice_0_prob' in predictions.columns and 'choice_1_prob' in predictions.columns:
        p=np.where(c==0,df.choice_0_prob,df.choice_1_prob)
        p=np.where(np.isnan(c),np.nan,p)
        log_p=np.log(p)
    else:
        raise ValueError('predictions must include either choice_0_log_prob and choice_1_log_prob or choice_0_prob and choice_1_prob')
    
    df['NLL']=-log_p
    
    # deal with trials for which we have no predictions:
    missing_predictions=np.logical_and(df['NLL'].isna(),np.logical_not(np.isnan(c))) # there's no prediction but the subject made a choice
    
    if np.any(missing_predictions):
        print('Found {} missing predictions. Handling strategy: {}.'.format(np.sum(missing_predictions),deal_with_missing_predictions))
        if deal_with_missing_predictions=='omit':
            df=df.loc[np.logical_not(missing_predictions),:] # drop these trails
        elif deal_with_missing_predictions=='uniform predictions':
            df[missing_predictions,'NLL']=-np.log(0.5) # make a uniform prediction
        else:
            raise ValueError("invalid deal_with_missing_predictions")
    
    # aggregate NLL
    if mode=='sum':
        agg_NLL=df.NLL.sum().item()        
    elif mode=='mean':
        agg_NLL=df.NLL.mean().item()
    if return_non_agg_NLL:
        return agg_NLL, df
    else:
        return agg_NLL

## Upper and lower bounds on the noise ceiling

In [9]:
def get_ub_noise_ceiling_predictions(subject_data):    
    # calculate human choice probability per sentence (collapsed over subjects and within-subject repetitions)
    df=subject_data.copy()
    df['chose_0']=np.asarray(subject_data.Choice==0,np.float)
    df['chose_0'].loc[np.isnan(subject_data.Choice)]==np.nan
    df2=df.groupby(['sentence1','sentence2','sentence_pair_id'])['chose_0'].mean()
    df2=df2.drop(columns=['subject']).reset_index().rename(columns={'chose_0':'choice_0_prob'})
    df2['choice_1_prob']=1.0-np.asarray(df2.choice_0_prob)
    return df2
ub_NC_predictions=get_ub_noise_ceiling_predictions(protocol)

def get_lb_noise_ceiling_predictions(subject_data,pseudo_counts=1.0):
    # calculate leave-one-subject out human choice probability per sentence (collapsed over subjects and within-subject repetitions)
    
    # pseudo_counts controls the symmetric beta prior (alpha=beta=pseudo_counts+1)
    subjects=list(np.unique(subject_data.subject))
    df1=subject_data.copy()
    df1['chose_0']=np.asarray(df1.Choice==0,np.float)
    df1['chose_0'].loc[np.isnan(df1.Choice)]==np.nan
    
    subject_specific_predictions_dfs=[]
    for subject in subjects: # leave one-subject out cross-validation:
        without_subject_df=df1.copy().loc[df1.subject!=subject,:] # hold out one subject
        if pseudo_counts!=0: # Bayesian prior
            choice_0_counts_without_subject=without_subject_df.groupby(['sentence1','sentence2','sentence_pair_id'])['chose_0'].sum()
            total_counts_without_subject=without_subject_df.groupby(['sentence1','sentence2','sentence_pair_id'])['chose_0'].count()
            subject_specific_predictions=(choice_0_counts_without_subject+pseudo_counts)/(total_counts_without_subject+pseudo_counts*2)
        else: # plain mean
            subject_specific_predictions=without_subject_df.groupby(['sentence1','sentence2','sentence_pair_id'])['chose_0'].mean() # mean choice probabilities of the remaining subjects                                    
        subject_specific_predictions=subject_specific_predictions.reset_index().rename(columns={'chose_0':'choice_0_prob'})
        subject_specific_predictions['choice_1_prob']=1.0-np.asarray(subject_specific_predictions.choice_0_prob)                
        subject_specific_predictions.loc[:,'subject']=subject # we mark the resulting predictions as subject-specific predictions
        
        # next, we remove predictions for sentence pairs the subject didn't see:
        cur_subject_df=df1.copy().loc[df1.subject==subject,:] 
        cur_subject_sentence_pair_ids=np.unique(cur_subject_df.sentence_pair_id) # these are the sentences the subject saw
        subject_specific_predictions=subject_specific_predictions.loc[np.in1d(subject_specific_predictions.sentence_pair_id,cur_subject_sentence_pair_ids),:]                
        subject_specific_predictions_dfs.append(subject_specific_predictions) # and add the remaining predictions to the list
    
    lb_NC_predictions=pd.concat(subject_specific_predictions_dfs)
    
    return lb_NC_predictions
lb_NC_predictions=get_lb_noise_ceiling_predictions(protocol)
lb_NC_NLL=evaluate_NLL(protocol,lb_NC_predictions,mode='sum')
print("lower noise ceiling NLL (before fitting)",lb_NC_NLL)


ub_NC_NLL=evaluate_NLL(protocol,ub_NC_predictions,mode='sum')
print("upper noise ceiling NLL",ub_NC_NLL)

# # without a Gamma parameter, the lower noise ceiling can be infinity
# lb_NC_NLL,df=evaluate_NLL(protocol,lb_NC_predictions,mode='sum',return_non_agg_NLL=True)
# print("lower noise ceiling NLL",lb_NC_NLL)

lower noise ceiling NLL (before fitting) 1685.7361062606578
upper noise ceiling NLL 1142.902361691926


In [10]:
ub_NC_predictions

,sentence1,sentence2,sentence_pair_id,choice_0_prob,choice_1_prob
0,A agony it Rapids and the bikini dipping,Most are the names of arrogance native land,A agony it Rapids and the bikini dipping_Most ...,0.00,1.00
1,A agony it Rapids and the bikini dipping,The Marina is a sailing boat from Scotland,A agony it Rapids and the bikini dipping_The M...,0.00,1.00
2,A and had to the past it again,The overpowering the lawn of such a year,A and had to the past it again_The overpowerin...,0.25,0.75
3,A case of driver replacements involved John Bowe,It includes notable people in the modern era,A case of driver replacements involved John Bo...,0.25,0.75
4,A civilization that was the largest weighing in,Talent beaches mirroring the publicly block tr...,A civilization that was the largest weighing i...,0.75,0.25
...,...,...,...,...,...
600,This tarnished devising of seconded cruelty an...,To retroactively your team of style and workout,This tarnished devising of seconded cruelty an...,0.25,0.75
601,Too much separates relax fabric choral rudder ...,Use it for this and many other things,Too much separates relax fabric choral rudder ...,0.00,1.00
602,Trust reflect slugging draped robots shoe firs...,Worth a turkey license issuance fortune choppe...,Trust reflect slugging draped robots shoe firs...,0.75,0.25
603,Was excited when I saw the awesome formatting,We were fools not to have known it,Was excited when I saw the awesome formatting_...,0.25,0.75


### Pseudo sentence log-probabilities
In order to deal with the noise ceiling estimates in the same fashion we deal with the deep nets' outputs, we convert their choice probabilities pseudo-sentence probabilities

In [11]:
def pseudo_sentence_probs(p_choice_1,p_choice_2,pseudo_sentence_prob_gamma=1.234,eps=1e-8):
    p_choice_1=np.clip(p_choice_1,eps,1-eps)
    p_choice_2=np.clip(p_choice_2,eps,1-eps)
    
    s1a_b=-np.log((1-p_choice_1)/p_choice_1)*pseudo_sentence_prob_gamma  
    lp1=-100+s1a_b/2
    lp2=-100-s1a_b/2

    return lp1,lp2

### Prepare model predictions

In [12]:
models=['bigram','trigram','rnn','lstm','bilstm','bert','bert_whole_word','roberta','xlm','electra','gpt2','ub_NC','lb_NC']

comp_models=list(np.setdiff1d(models,['ub_NC','lb_NC']))
human_models=list(np.setdiff1d(models,comp_models))

def get_cached_sentence_log_probabilities_for_model(model_name,protocol):
    """ build a pandas dataframe of model raw sentence probabilities using cached model outputs"""
    f = open(model_name+'_expt1_sentence_probs.pkl','rb')
    prob_dict=pickle.load(f)
    model_predictions=protocol[['sentence1','sentence2','sentence_pair_id']].drop_duplicates().reset_index(drop=True) 
    for i_row, row in model_predictions.iterrows():
        model_predictions.at[i_row,'sentence1_raw_log_prob']=prob_dict[row['sentence1']]
        model_predictions.at[i_row,'sentence2_raw_log_prob']=prob_dict[row['sentence2']]
    return model_predictions

def noise_ceiling_to_pseudo_log_probabilities(NC_predictions):
    """ recast trial-specific noise ceiling predictions to be language-model like """
    NC_predictions=NC_predictions.copy()
    p1=NC_predictions['choice_0_prob']
    p2=NC_predictions['choice_1_prob']
    assert np.isfinite(p1).all() and np.isfinite(p2).all(), 'found nan predictions for subject {}: {}).'.format(trial.subject,trial.sentence_pair_id)
    log_p1,log_p2=pseudo_sentence_probs(p1,p2)
    NC_predictions['sentence1_raw_log_prob']=log_p1
    NC_predictions['sentence2_raw_log_prob']=log_p2
    NC_predictions.drop(columns=['choice_0_prob','choice_1_prob'])
    return NC_predictions

model_sentence_predictions=OrderedDict() # this will hold a dataframe for each model

for model_name in models:
    if model_name in comp_models:
        model_sentence_predictions[model_name]=get_cached_sentence_log_probabilities_for_model(model_name,protocol)
    elif model_name=='ub_NC':
        model_sentence_predictions[model_name]=noise_ceiling_to_pseudo_log_probabilities(ub_NC_predictions)
    elif model_name=='lb_NC':
        model_sentence_predictions[model_name]=noise_ceiling_to_pseudo_log_probabilities(lb_NC_predictions)
    else:
        raise ValueError

In [13]:
# an example:
model_sentence_predictions['lb_NC']

,sentence1,sentence2,sentence_pair_id,choice_0_prob,choice_1_prob,subject,sentence1_raw_log_prob,sentence2_raw_log_prob
9,A naturally occurring replies punishment exami...,This country uses collaborative ping internal ...,A naturally occurring replies punishment exami...,0.166667,0.833333,0,-100.993023,-99.006977
10,A poets piped enquiry a lot of it,The growing spectral and thus attain the bandit,A poets piped enquiry a lot of it_The growing ...,0.500000,0.500000,0,-100.000000,-100.000000
11,A poster and employer approval rating and vows,The next to the time and historical park,A poster and employer approval rating and vows...,0.666667,0.333333,0,-99.572328,-100.427672
37,Also their navy is in a dismal state,He had never told us he was married,Also their navy is in a dismal state_He had ne...,0.500000,0.500000,0,-100.000000,-100.000000
46,And also as used in the plant printers,Hank rusted transcended youthful tiles manslau...,And also as used in the plant printers_Hank ru...,0.666667,0.333333,0,-99.572328,-100.427672
...,...,...,...,...,...,...,...,...
582,The overall growth rate of the damned thing,There are notoriously marshes and fatally simu...,The overall growth rate of the damned thing_Th...,0.600000,0.400000,23,-99.749828,-100.250172
594,There has been huge receptions at every gate,They were to go there on a coach,There has been huge receptions at every gate_T...,0.400000,0.600000,23,-100.250172,-99.749828
595,There is a chemical imbalance in our brains,Under Abbot it never has and never will,There is a chemical imbalance in our brains_Un...,0.800000,0.200000,23,-99.144656,-100.855344
596,These tables were lists of those who participate,Turnpike Gutierrez at newcomer golfers may ste...,These tables were lists of those who participa...,0.666667,0.333333,23,-99.572328,-100.427672


In [57]:
def fit_model_decision_parameters(protocol,model_predictions,decision_model_class='FixedWidthSquashing',device='cpu',optimizer='LBFGS',initial_squash_parameter=None,initial_gamma_parameter=None,max_steps_without_improvement=10,return_dataframes=True,verbose=True,param_grid=None,frozen_parameters=None):
    """ fit the model decision hyper-parameter given a human data (protocol) and sentence predictions 
    
    args:
    protocol (pd.dataframe) must include a sentence_pair_id column
    model_prediction (pd.dataframe) must include a sentence_pair_id column as well as sentence1_raw_log_prob and sentence2_raw_log_prob
    decision_model_class (str) name of class to import from model_to_human_decision_torch. Currently implemented: 'Naive', 'NoSquashing', 'FixedWidthSquashing', 'VariableWidthSquashing', 'SquashedSoftmax'
    device (str) torch device. cpu is faster for float64
    optimizer (str) 'LBFGS' or 'Adam'
    initial_squash_parameter (float) initial value for the squashing parameter 
    initial_gamma_parameter (float) initial value for the gamma parameter
    max_steps_without_improvement (int) when to stop when not improving
    return_dataframes (boolean) see below
    verbose (boolean) show messages
    param_grid (dict) if provided, uses sklearn.model_selection.ParameterGrid to return the model with the best NLL
    frozen_parameters (list of strings) paramters to freeze (not optimize)
    
    returns:
    if return_dataframes is True:
        decision_model (ModelToHumanDecision) a fitted decision model object
        parameters_dict (dict) the fitted parameters
        protocol_with_NLL (pd.dataframe) the protocol with additional NLL column (trial-specific NLL)
        adjusted_model_predictions (pd.dataframe) the model predictions with additional columns for human choice probabilities
    otherwise:
        decision_model, parameters_dict        
    """    
    
    # optional grid search
    if param_grid is not None:
        def eval_NLL_with_params(params):
            kwargs={'decision_model_class':decision_model_class,'device':device,'optimizer':optimizer,'initial_squash_parameter':initial_squash_parameter,
                    'initial_gamma_parameter':initial_gamma_parameter,
                    'max_steps_without_improvement':max_steps_without_improvement,'verbose':verbose,
                    'return_dataframes':return_dataframes,'frozen_parameters':frozen_parameters}
            kwargs.update(params)
            return fit_model_decision_parameters(protocol,model_predictions,**kwargs)                    
        NLL=list()
        for params in sklearn.model_selection.ParameterGrid(param_grid):
            params_=params.copy()
            params_['verbose']=False
            params_['return_dataframes']=False
            cur_NLL=eval_NLL_with_params(params_)[2]
            if verbose:
                print(params,':',cur_NLL,'NLL')
            NLL.append(cur_NLL)
        best_i=np.argmin(NLL)
        best_params=sklearn.model_selection.ParameterGrid(param_grid)[best_i]
        if verbose:
            print('best params:',best_params)        
        return eval_NLL_with_params(best_params)
        
    if 'subject' in model_predictions.columns:
        identifiers=['subject','sentence1','sentence2','sentence_pair_id']
    else:
        identifiers=['sentence1','sentence2','sentence_pair_id']
        
    merged=protocol.merge(model_predictions,on=identifiers)  
    
    log_p1=torch.tensor(merged['sentence1_raw_log_prob'],device=device,dtype=torch.float64)
    log_p2=torch.tensor(merged['sentence2_raw_log_prob'],device=device,dtype=torch.float64)
    response_ind=torch.tensor(merged['Choice'],device=device,dtype=int)
    
    parameters={}
    if initial_squash_parameter is None:
        if decision_model_class in ['FixedWidthSquashing','VariableWidthSquashing']:
            # get minimal sentence log-probability to serve as initial value for the squashing parameters
            minps=abs(min(log_p1.min(),log_p2.min()))            
            parameters['squashes']=minps
    else:    
            parameters['squashes']=initial_squash_parameter
    
    if initial_gamma_parameter is not None:
        parameters['gamma']=initial_gamma_parameter
    
    # build decision model object
    decision_model=getattr(model_to_human_decision_torch,decision_model_class)(parameters=parameters,device=device)
    
    for par in decision_model.parameters():
        par.requires_grad=True
    
    if frozen_parameters is not None:
        parameters_to_optimize=dict(decision_model.named_parameters())
        for frozen_parameter in frozen_parameters:
            del parameters_to_optimize[frozen_parameter]
        parameters_to_optimize=parameters_to_optimize.values()
    else:
        parameters_to_optimize=decision_model.parameters()
            
    if optimizer == 'Adam':
        opt = optim.Adam(parameters_to_optimize,lr=1.0)
    elif optimizer=='LBFGS':
        opt = optim.LBFGS(parameters_to_optimize,line_search_fn='strong_wolfe') # this is much faster!
    else:
        raise ValueError
      
    previous_loss=torch.tensor(np.inf,device=device)
   
    eps=1e-10
    non_improvement_counter=0
    t=time.time()
    for epoch in range(1000):

        opt.zero_grad()
        
        def calc_loss():
            global my_loss
            log_p_sent=decision_model(log_p1,log_p2)
            take_by_2nd_dim=lambda x, idx: x[torch.arange(x.size(0)), idx] 
            log_p_choice=take_by_2nd_dim(log_p_sent,response_ind)
            loss=log_p_choice.sum()     
            return loss
        
        def closure(): # required for LBFGS
            opt.zero_grad()
            loss=calc_loss()
            loss.backward()            
            return loss
    
        opt.step(closure)
    
        loss=calc_loss().detach()
        
        if (previous_loss-loss)<eps:
            non_improvement_counter+=1            
        else:            
            non_improvement_counter=0        
        previous_loss=loss.detach()
        
        if non_improvement_counter>=max_steps_without_improvement:
            if verbose:
                print('converged in',epoch,'epochs,',time.time()-t,'seconds')
            break
            
        if epoch%50==0 and verbose:            
            print('epoch: ', epoch, 'loss:',loss.item())
            # TODO: show parameters

            #     print(decision_model.get_parameters())
#     print('total loss:',loss)
#     for i_model,model_name in enumerate(models):
#         print('{} {:.2f}'.format(model_name,model_loss[i_model].item()))
        
    #breakpoint()
    if verbose:
        print("final loss ",loss.item())

    parameters_dict=decision_model.get_parameters()
    
    NLL=loss.item()
    
    if return_dataframes:        
        # trial-specific NLL
        log_p_sent=decision_model(log_p1,log_p2)
        take_by_2nd_dim=lambda x, idx: x[torch.arange(x.size(0)), idx] 
        log_p_choice=take_by_2nd_dim(log_p_sent,response_ind)
            
        protocol_with_NLL=protocol.copy()
        protocol_with_NLL['NLL']=log_p_choice.detach().cpu().numpy()
        adjusted_model_predictions=model_predictions.copy()
        
        with torch.no_grad():
            decision_log_prob=-decision_model(model_predictions.filter(regex='_raw_log_prob$',axis=1).to_numpy()).detach().cpu().numpy()
        adjusted_model_predictions['choice_0_log_prob']=decision_log_prob[:,0]
        adjusted_model_predictions['choice_1_log_prob']=decision_log_prob[:,1]
        adjusted_model_predictions=adjusted_model_predictions.drop(columns=['sentence1_raw_log_prob','sentence2_raw_log_prob','choice_0_prob','choice_1_prob'],errors='ignore')        
        return decision_model, parameters_dict, NLL, protocol_with_NLL, adjusted_model_predictions
    else:
        return decision_model, parameters_dict, NLL

### Compare between optimizers

In [58]:
optimizers=['LBFGS','Adam']
for model in comp_models:        
    print('**** fitting '+model + ' ****')
    param_grid=[
                {'decision_model_class':['FixedWidthSquashing'],'optimizer':['Adam']}, # use minps
                {'decision_model_class':['FixedWidthSquashing'],'optimizer':['LBFGS'],}, # use minps
                {'decision_model_class':['FixedWidthSquashing'],'optimizer':['LBFGS'],'initial_squash_parameter':np.linspace(0,300,31)},
                #{'decision_model_class':['SquashedSoftmax'],'optimizer':optimizers},            
                #{'decision_model_class':['FixedWidthSquashing'],'initial_squash_parameter':np.linspace(0,300,31)},                    
                #{'decision_model_class':['VariableWidthSquashing'],'initial_squash_parameter':np.linspace(0,300,31)},
                #{'decision_model_class':['VariableWidthSigmoid'],'initial_squash_parameter':np.linspace(0,300,31)},
    ]

    decision_model, parameters_dict, NLL, protocol_with_NLL, adjusted_model_predictions=fit_model_decision_parameters(protocol,model_sentence_predictions[model],param_grid=param_grid,verbose=True)
    print(parameters_dict)
    print(' ---')

**** fitting bert ****


KeyboardInterrupt: 

In [82]:
# some sanity checks:
assert np.isclose(NLL,evaluate_NLL(protocol,adjusted_model_predictions,mode='sum'))
assert np.isclose(NLL,protocol_with_NLL['NLL'].sum())

### Compare between decision models

In [90]:
optimizers=['LBFGS']
for model in comp_models:        
    print('**** fitting '+model + ' ****')
    param_grid=[                
                {'decision_model_class':['NoSquashing'],'optimizer':optimizers,}, # use minps
                {'decision_model_class':['FixedWidthSquashing'],'optimizer':optimizers,}, # use minps
                {'decision_model_class':['SquashedSoftmax'],'optimizer':optimizers},      
        
                #{'decision_model_class':['FixedWidthSquashing'],'initial_squash_parameter':np.linspace(0,300,31)},                    
                #{'decision_model_class':['VariableWidthSquashing'],'initial_squash_parameter':np.linspace(0,300,31)},
                #{'decision_model_class':['VariableWidthSigmoid'],'initial_squash_parameter':np.linspace(0,300,31)},
    ]

    decision_model, parameters_dict, NLL, protocol_with_NLL, adjusted_model_predictions=fit_model_decision_parameters(protocol,model_sentence_predictions[model],param_grid=param_grid,verbose=True)
    print(parameters_dict)
    print(' ---')

**** fitting bert ****
{'decision_model_class': 'NoSquashing', 'optimizer': 'LBFGS'} : 1571.4023663341602 NLL
{'decision_model_class': 'FixedWidthSquashing', 'optimizer': 'LBFGS'} : 1571.2075436385578 NLL
{'decision_model_class': 'SquashedSoftmax', 'optimizer': 'LBFGS'} : 1565.5173837412353 NLL
best params: {'optimizer': 'LBFGS', 'decision_model_class': 'SquashedSoftmax'}
epoch:  0 loss: 1565.5173837412353
converged in 10 epochs, 0.0848989486694336 seconds
final loss  1565.5173837412353
{'gamma': 22.669362863671388, 'squashes': -4.374073384455098}
 ---
**** fitting bert_whole_word ****
{'decision_model_class': 'NoSquashing', 'optimizer': 'LBFGS'} : 1572.7144879139698 NLL
{'decision_model_class': 'FixedWidthSquashing', 'optimizer': 'LBFGS'} : 1572.5825426301844 NLL
{'decision_model_class': 'SquashedSoftmax', 'optimizer': 'LBFGS'} : 1568.83091150882 NLL
best params: {'optimizer': 'LBFGS', 'decision_model_class': 'SquashedSoftmax'}
epoch:  0 loss: 1568.83091150882
converged in 10 epochs, 

In [98]:
# train and save decision models for all models
optimizer='LBFGS'
sent_len=8

# train and save models
for decision_model_class in ['SquashedSoftmax','FixedWidthSquashing','NoSquashing']:
    for model in comp_models:
        if decision_model_class=='FixedWidthSquashing':
            param_grid=[{'initial_squash_parameter':np.linspace(0,300,31)}]
        else:
            param_grid=None
        decision_model, parameters_dict, NLL, protocol_with_NLL, adjusted_model_predictions=fit_model_decision_parameters(protocol,model_sentence_predictions[model],decision_model_class=decision_model_class,optimizer=optimizer,verbose=False,param_grid=param_grid)
        path = os.path.join('decision_models','20210115',decision_model_class+"_" +optimizer+"_{}_word".format(sent_len),model+'.pkl')
        print(model,decision_model_class,'NLL:',NLL,'parameters:',parameters_dict)
        decision_model.save(path)

bert SquashedSoftmax NLL: 1565.5173837412353 parameters: {'gamma': 22.669362863671388, 'squashes': -4.374073384455098}
bert_whole_word SquashedSoftmax NLL: 1568.83091150882 parameters: {'gamma': 26.37773967620515, 'squashes': -3.8529237077746337}
bigram SquashedSoftmax NLL: 1560.3624454805104 parameters: {'gamma': 22.2896674367132, 'squashes': -7.275876341461244}
bilstm SquashedSoftmax NLL: 1564.2242626652098 parameters: {'gamma': 22.976206767245323, 'squashes': -4.0391831417504145}
electra SquashedSoftmax NLL: 1553.322107184143 parameters: {'gamma': 22.70907833066752, 'squashes': -3.9010957421553853}
gpt2 SquashedSoftmax NLL: 1554.866205093827 parameters: {'gamma': 23.710766043078475, 'squashes': -4.027735109878557}
lstm SquashedSoftmax NLL: 1555.1082232327792 parameters: {'gamma': 24.102962008102438, 'squashes': -3.6639694072535454}
rnn SquashedSoftmax NLL: 1569.6613433861287 parameters: {'gamma': 23.76740230289882, 'squashes': -3.7020861415349304}
roberta SquashedSoftmax NLL: 1553.8

In [81]:
# train and save FixedWidthSquashing decision models using percentile based squashing threshold
optimizer='LBFGS'
sent_len=8

which_percentile_to_use=10
model_logprob_percentiles_dict=pickle.load(open('model_logprob_percentiles.pkl','rb'))

# train and save models
for decision_model_class in ['FixedWidthSquashing']:
    for model in comp_models:        
        squashing_threshold=abs(model_logprob_percentiles_dict[model][which_percentile_to_use])
        decision_model, parameters_dict, NLL, protocol_with_NLL, adjusted_model_predictions=fit_model_decision_parameters(
            protocol,model_sentence_predictions[model],decision_model_class=decision_model_class,
            optimizer=optimizer,verbose=False,
            frozen_parameters=['squashes'],initial_squash_parameter=squashing_threshold)
        path = os.path.join('decision_models','20210118_10th_percentile_squash',decision_model_class+"_" +optimizer+"_{}_word".format(sent_len),model+'.pkl')
        print(model,decision_model_class,'NLL:',NLL,'parameters:',parameters_dict)
        decision_model.save(path)

bert FixedWidthSquashing NLL: 1658.2361335345831 parameters: {'squashes': 62.71453811046773, 'gamma': 11.384543899159606}
bert_whole_word FixedWidthSquashing NLL: 1652.2034728378417 parameters: {'squashes': 56.92383811131959, 'gamma': 12.950105725448667}
bigram FixedWidthSquashing NLL: 1643.8402985992554 parameters: {'squashes': 121.51904296875, 'gamma': 12.341410286904521}
bilstm FixedWidthSquashing NLL: 1656.0935858463858 parameters: {'squashes': 57.18711783290085, 'gamma': 11.845979709762771}
electra FixedWidthSquashing NLL: 1647.5160489652053 parameters: {'squashes': 49.83518087675971, 'gamma': 11.127596031582593}
gpt2 FixedWidthSquashing NLL: 1675.619378669754 parameters: {'squashes': 48.35059190027987, 'gamma': 10.512459597557543}
lstm FixedWidthSquashing NLL: 1644.7726118109154 parameters: {'squashes': 56.265304489099634, 'gamma': 12.779863227391383}
rnn FixedWidthSquashing NLL: 1653.0387932698545 parameters: {'squashes': 60.962956533614665, 'gamma': 14.849769912840127}
roberta 

In [ ]:
get_parameters_across_models(decision_model)

In [ ]:
gT=np.array(gamma_boots).T
sT=np.array(squash_boots).T

ccs=[]
for i in range(11):
    
    cc=np.corrcoef(sT[i,:],gT[i,:])[0,1]
    ccs.append(cc)
    
ccs




In [ ]:
#synthetic

# tensor(671.4779, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
# tensor([129.5626, 128.7844,  84.1193,  92.4041, 111.8108, 105.3416,  83.8154,
#         116.5818,  87.0363,  60.6492,  81.0560], device='cuda:0',
#        dtype=torch.float64, requires_grad=True)
# tensor([29.8585, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,
#         10.0000, 10.0000, 10.0000], device='cuda:0', requires_grad=True)


#natural

# tensor(148.7837, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
# tensor([115.8415,  84.2844,  72.0876,  43.8699,  54.9998,  50.4821,  69.6983,
#          49.1065,  42.4528,  34.6840,  59.5816], device='cuda:0',
#        dtype=torch.float64, requires_grad=True)
# tensor([ 9.1418, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,
#         10.0000, 10.0000, 10.0000], device='cuda:0', requires_grad=True)


# tensor(829.6835, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
# tensor([129.1968, 127.7177,  83.4297,  91.0015, 111.4391, 104.3089,  83.3092,
#         115.3282,  85.6379,  59.8025,  79.7266], device='cuda:0',
#        dtype=torch.float64, requires_grad=True)
# tensor([28.2664, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,
#         10.0000, 10.0000, 10.0000], device='cuda:0', requires_grad=True)


In [ ]:
# squash_boots=np.array([[136.6031, 130.6315,  88.2857, 127.1014, 124.6718, 110.4878,  84.6417,
#         118.1435, 107.7737,  63.0790,  79.9143],
#               [137.9003, 125.7105,  84.5635, 127.1772, 112.0871,  94.5774,  78.6344,
#         114.0863,  95.4273,  58.2541,  79.6323],
#               [131.8761, 133.7181, 147.7142, 143.2530, 110.6293, 134.2203,  83.7578,
#         139.0220,  82.7407,  66.7261,  79.2919],
#               [139.6443, 135.6945,  80.5810,  94.8058, 116.6113, 103.0531,  87.4392,
#         119.7775,  81.2322,  60.9193,  81.1773],
#               [132.4967, 128.7608,  92.4549,  91.8489, 110.4346, 104.1544,  80.5317,
#         126.0785, 124.9601,  57.4880,  76.9285],
#               [128.2988, 112.9426,  80.0811,  90.9844, 111.0652, 115.8396,  56.3730,
#          78.9889, 120.2621,  29.6331,  61.9582],
#               [127.7181, 132.9688, 148.8201,  94.7272, 109.7225,  87.5404,  81.3132,
#         115.8013, 116.9241,  64.7583,  73.5550],
#               [120.8540, 116.8421, 135.4461,  88.5892, 124.1079, 133.7935,  86.1704,
#         100.1683,  92.7386,  49.7565,  79.4477],
#               [125.4391, 114.5180,  82.5459,  87.4416, 110.8198,  93.8739,  63.4568,
#         117.9814,  83.9064,  57.2624,  77.2107],
#               [129.6447, 129.3590,  86.6444,  92.5276, 109.5141, 105.0256,  65.8152,
#         117.5682, 125.8277,  59.4612,  77.8789]])

s=squash_boots

squash_boots=np.array(losses).T
squash_boots.sort()
squash_boots=squash_boots.T

In [ ]:
#squash_boots=squash_boots.reshape([10,11])



x=np.arange(11)*2
# plt.bar(x-0.2, maxps, width=0.1, color=[1,0,0])
plt.bar(x-0.1, squash_boots[0], width=0.1, color=[0,0,1])
plt.bar(x, squash_boots[1], width=0.1, color=[0,0,1])
plt.bar(x+0.1, squash_boots[2], width=0.1, color=[0,0,1])
plt.bar(x+0.2, squash_boots[3], width=0.1, color=[0,0,1])
plt.bar(x+0.3, squash_boots[4], width=0.1, color=[0,0,1])
plt.bar(x+0.4, squash_boots[5], width=0.1, color=[0,0,1])
plt.bar(x+0.5, squash_boots[6], width=0.1, color=[0,0,1])
plt.bar(x+0.6, squash_boots[7], width=0.1, color=[0,0,1])
plt.bar(x+0.7, squash_boots[8], width=0.1, color=[0,0,1])
plt.bar(x+0.8, squash_boots[9], width=0.1, color=[0,0,1])
# plt.bar(x+0.9, minps, width=0.1, color=[1,0,0])

# plt.ylabel('Squash Thresholds')
plt.ylabel('Negative log likelihood')
plt.xticks(x,models,rotation=70)

# plt.ylim([800,950])

legs=['Min/Max baseline','Squash thresholds']
# plt.legend(legs,bbox_to_anchor=(1, 0.3, 0.2, 0.5))

In [ ]:
squash_boots

In [ ]:
sT

In [ ]:
d=squash_boots.T

In [ ]:

d=np.array([f.sort() for f in d])

In [ ]:
d

In [ ]:
squash_boots=d.T

In [ ]:
slist=list(set(all_sents))

In [ ]:
file=open('test_sents_expt1_first13subs.txt','w')
for s in slist:
    file.write(s)
    file.write('\n')

In [ ]:
len(slist)

In [ ]:
decision_model.parameters['squashes'].data.numpy()


In [ ]:
np.log(0.9)

In [ ]:
[sent1,sent2].sort()

In [ ]:
g=0
a=0
for trial in fitset:
    
    if trial[0]=='electra':
        
        a+=1
    
        pa=trial[1]
        pb=trial[2]
        c=trial[3]

        if c==0 and pa>pb:
            g+=1
        elif c==1 and pb>pa:
            g+=1


In [ ]:
g/a

In [ ]:
g

In [ ]:
g/len(fitset)

In [ ]:
np.log(.66)

In [ ]:
fitsets

In [ ]:
fitset

In [ ]:
np.log(item_prob_ceilings[item_id][1])==-math.inf

In [ ]:
fitset